# Python多任务处理(协程篇)
作者: tushushu  
项目地址: https://github.com/tushushu/flying-python

## 1. 协程
协程，英文名是Coroutine，又称为微线程，是一种用户态的轻量级线程。协程不像线程和进程那样，需要进行系统内核上的上下文切换，协程的上下文切换是由程序员决定的。协程通过 async/await 语法进行声明，是编写异步应用的推荐方式。


In [1]:
import asyncio

async def hello_world():
    print('hello')
    await asyncio.sleep(1)
    print('world')

# asyncio.run(main())

## 2. 协程处理IO密集型任务
IO密集型任务指的是系统的CPU性能相对硬盘、内存要好很多，此时，系统运作，大部分的状况是CPU在等I/O (硬盘/内存) 的读/写操作，此时CPU Loading并不高。涉及到网络、磁盘IO的任务都是IO密集型任务。

In [1]:
import requests
import time

### 2.1 定义一个IO密集型函数

In [2]:
def _sleep(n):
    time.sleep(n)
    print("Sleep for %d seconds." % n)

### 2.2 使用串行的方式处理

In [3]:
def sleep(m, n):
    for i in range(m):
        _sleep(n)

### 2.3 使用协程处理
目前Jupyter Notebook因为一些待修复的BUG无法运行协程。

In [4]:
f = open("tmp.py", "w")
f.write(
"""
import time
import asyncio


async def _sleep(n):
    await asyncio.sleep(n)
    print("Sleep for %d seconds." % n)


def sleep(m, n):
    loop = asyncio.get_event_loop()
    loop.run_until_complete(asyncio.gather(*[_sleep(n) for _ in range(m)]))
    loop.close()


if __name__ == '__main__':
    start = time.perf_counter()
    m = 3
    n = 1
    sleep(m, n)
    print("%s函数的运行时间为 %.5f 秒!" % (sleep.__name__, time.perf_counter() - start))"""
)
f.close()

### 2.4 计算函数运行时间
- 串行版本的运行时间 = 1 + 2 + 3 = 6秒  
- 多线程版本的运行时间 = max(1, 2, 3) = 3秒

In [5]:
def time_it(fn, *args):
    start = time.perf_counter()
    fn(*args)
    print("%s函数的运行时间为 %.5f 秒!" % (fn.__name__, time.perf_counter() - start))

In [6]:
time_it(sleep, 3, 1)

Sleep for 1 seconds.
Sleep for 1 seconds.
Sleep for 1 seconds.
sleep函数的运行时间为 3.01054 秒!


In [7]:
import subprocess
print(str(subprocess.check_output("python tmp.py", shell=True), encoding = "utf-8"))

Sleep for 1 seconds.
Sleep for 1 seconds.
Sleep for 1 seconds.
sleep函数的运行时间为 1.00305 秒!



## 参考文章
https://docs.python.org/zh-cn/3.7/library/asyncio-task.html